<a href="https://colab.research.google.com/github/albertodiez1980LaRioja/Prueba/blob/master/ConsultaNotaPorComentario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
#@title Pulsa en la flecha de la izquierda para empezar:


import warnings
warnings.filterwarnings('ignore')

!pip install fastai2 -q

from fastai2.text.all import *


print("Cargando los modelos, espere...")

# descargamos y descomprimimos el modelo
!wget https://www.dropbox.com/s/jlgc1b5epklhl8m/finetuned3.pth?dl=0 -O finetuned3.pth --quiet

!wget https://www.dropbox.com/s/ymnlbpxkkh99qqu/Train.zip?dl=0 -O Train.zip --quiet
!unzip -qq -o Train.zip

!wget https://www.dropbox.com/s/0h3y720qp4hkyg7/Test.zip?dl=0 -O Test.zip --quiet
!unzip -qq -o Test.zip

get_imdb = partial(get_text_files, folders=['Train', 'Test'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder('.', is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders('.', path='.', bs=128, seq_len=80)

dls_clas = DataBlock(
    blocks=(TextBlock.from_folder('.', vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['Train', 'Test']),
    #splitter=GrandparentSplitter(valid_name='Test')
).dataloaders('.', path='.', bs=128, seq_len=72)
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 

                                metrics=accuracy).to_fp16()

learn = learn.load_encoder('../finetuned3')

print("Modelos cargados")

import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Pulsa para clasificar")
output = widgets.Output()
texto=widgets.Text(value='Pon aquí la URL', disabled=False)

def on_button_clicked(b):
  # Display the message within the output widget.
  if(b.description=='Pulsa para clasificar'):
    with output:
      print("Espere a que la clasificación sea calculada...")
      pred_class,pred_idx,outputs=learn.predict(texto.value)
      print("La nota calculada es: "+pred_class)
button.on_click(on_button_clicked)

display(texto)
display(button, output)


     |████████████████████████████████| 194kB 4.7MB/s 
Cargando los modelos, espere...


NameError: ignored